In [1]:
import keras
import tensorflow as tf

# Memory Pre-configuration
config = tf.compat.v1.ConfigProto(
    gpu_options=tf.compat.v1.GPUOptions(
        per_process_gpu_memory_fraction=0.9,
        allow_growth = True
    )
    # device_count = {'GPU': 1}
)
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

지금까지는 **Sequential** 모델을 사용하여 만들었습니다.
* 네트워크 입력과 출력이 하나라고 가정합니다
* 이 모델은 층을 차례대로 쌓아 구성합니다

<br></br>
![](../images/7-1-sequential.png)
<br></br>

일부 네트워크는 *개별 입력이 여러개* 필요하거나 *출력이 여러개* 필요합니다. 층을 여러개로 쌓지 않고 층 사이를 연결하여 그래프처럼 만드는 네트워크도 있습니다.

예를 들어 어떤 작업은 **다용한 종류의 입력**이 필요합니다. 다양한 입력 소스에서 전달된 데이터를 다른 종류의 신경망 층을 사용하여 처리하고 합칩니다.
* 중고 의류의 시장 가격을 에측하는 모델
* 사용자가 제공한 메타데이터 (브랜드, 연도 등), 텍스트 설명, 제품 사진을 입력으로 사용
* 메타데이터만 있다면 원-핫 인코딩으로 바꾸고 완전 연결 네트워크를 사용
* 텍스트 설명만 있다면 RNN이나 1D ConvNet을 사용
* 사진 이미지만 있다면 2D ConvNet 사용

세가지 모델을 모두 사용할 수 있을까요?
* 간단한 방법은 각자 훈련하고 각 예측을 가중 평균 **(weighted average)**을 계산
* 각 모델에서 추출하는 정보가 중복된다면 이는 최적의 방식이 아님
* 입력 데이터를 **동시에 사용**해서 정확한 하나의 모 델을 학습하는 것이 더 나은 방법
* 이 모델은 3개의 **입력 가지 (branch)**가 필요합니다

<br></br>
![](../images/7-1-multiinput.png)
<br></br>

이와 비슷하게 어떤 작업은 입력 데이터에서 **여러개의 타깃 속성**을 예측해야 합니다.
* 예를 들어 소설이나 짧은 글이 있을때 자동으로 장르별로 분류하고 글이 쓰여진 대략의 시대를 예측해야 합니다
* 물론 2개의 모델을 따로 훈련할 수 있습니다. (장르를 위한 모델 1개와 시대를 위한 모델 1개)
* 하지만 이 속성들은 *통계적으로 독립이지 않기때문에* 동시에 장르와 시대를 함께 예측하도록 학습해야 더 좋은 모델을 만들수 있습니다
* 이 모델은 2개의 출력 또는 **머리 (head)**를 가집니다.
* 장르와 시대 사이의 상관관계 때문에 소설의 시대를 알면 장르의 공간에서 정확하고 풍부한 표현을 학습하는데 도움이 됩니다

<br></br>
![](../images/7-1-multioutput.png)
<br></br>

최근에 개발된 많은 신경망 구조는 선형적이지 않은 네트워크 토폴로지 (topology)가 필요합니다.
* 비순환 유향 그래프 같은 구조
* **Inception Module**을 사용하는 **인셉션** 계열의 네트워크
* 이 모듈에서 입력은 나란히 놓은 여러개의 합성곱 층을 거쳐 하나의 텐서로 출력이 합쳐집니다

<br></br>
![](../images/7-1-inception.png)
<br></br>

최근에는 모델에 **잔차 연결**을 추가하기도 합니다. 잔차 연결은 하위 층의 출력 텐서를 상위 층의 출력 텐서에 더해져서 아래층의 표현이 네트워크 위쪽으로 흘러갈 수 있도록 합니다. 하위 층에서 학습된 정보가 데이터 처리 과정에서 손실되는 것을 막아줍니다.

<br></br>
![](../images/7-1-connected.png)
<br></br>

### 함수형 API

**함수형 API (Functional API)**에서는 직접 텐서들의 입출력을 다룹니다. 함수처럼 층을 사용하여 텐서를 입력받고 출력합니다.

```python
from keras import Input, layers

## 입력 텐서
input_tensor = Input(shape(32,))

## 함수처럼 사용하기 위해 층 객체를 만듭니다
dense = layers.Dense(32, activation = "relu")

## 텐서와 함께 층을 호출하면 텐서를 반환합니다
output_tensor = dense(input_tensor)
```

간단한 예를 통해 Sequential 모델과 함수형 API로 만든 동일한 모델을 비교해보겠습니다.

In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation = "relu", input_shape = (64,)))
seq_model.add(layers.Dense(32, activation = "relu"))
seq_model.add(layers.Dense(10, activation = "softmax"))

input_tensor = Input(shape = (64,))
x = layers.Dense(32, activation = "relu")(input_tensor)
x = layers.Dense(32, activation = "relu")(x)
output_tensor = layers.Dense(10, activation = "softmax")(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


모델 객체를 사용한 컴파일, 훈련, 평가 API는 Sequential 클래스와 같습니다.

In [3]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs = 10, batch_size = 128)

score = model.evaluate(x_train, y_train)
score

Epoch 1/10
8/8 [==============================] - 1s 1ms/step - loss: 12.0232
Epoch 2/10
8/8 [==============================] - 0s 1ms/step - loss: 12.9709
Epoch 3/10
8/8 [==============================] - 0s 1ms/step - loss: 14.7187
Epoch 4/10
8/8 [==============================] - 0s 1ms/step - loss: 17.0901
Epoch 5/10
8/8 [==============================] - 0s 1ms/step - loss: 20.1986
Epoch 6/10
8/8 [==============================] - 0s 829us/step - loss: 23.2866
Epoch 7/10
8/8 [==============================] - 0s 1ms/step - loss: 27.2852
Epoch 8/10
8/8 [==============================] - 0s 1ms/step - loss: 32.2790
Epoch 9/10
8/8 [==============================] - 0s 1ms/step - loss: 37.4013
Epoch 10/10
32/32 [==============================] - 0s 678us/step - loss: 49.2779


49.27785110473633

### 다중 입력 모델

함수형 API는 다중 입력 모델을 만드는데 사용할 수 있습니다. 
* 서로 다른 입력 가치를 합치기 위해 여러 텐서를 연결할 수 있는 층을 사용합니다. 
* 텐서를 더하거나 이어 붙이는 식입니다
* `keras.layers.add`, `keras.layers.concatenate` 등입니다

아주 간단한 다중 입력 모델을 살펴보겠습니다. Question-Answering 모델입니다.
* 전형적인 QA 모델은 2개의 입력을 가집니다.
* 한개는 **자연어 질문**이고 하나는 **답변에 필요한 정보가 담겨진 텍스트**입니다
* 모델은 답을 출력합니다
* 가장 간단한 구조는 미리 정의한 어휘 사전에서 소프트맥스 함수를 통한 한 단어로 된 답을 출력하는 것입니다.

<br></br>
![](../images/7-1-qa.png)
<br></br>

함수형 API를 사용하여 이런 모델을 만들어 보겠습니다. 
* 텍스트와 질문을 벡터로 인코딩하여 독립된 입력 2개를 정의합니다
* 그 다음 이 벡터를 연결하고 그 위에 소프트맥스 분류기를 추가합니다

##### 2개의 입력을 가진 QA 모델의 함수형 API 구현

In [4]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

## 텍스트 입력은 길이가 정해지지 않은 정수 시퀀스
text_input = Input(shape = (None,), dtype = 'int32', name = 'text')

## 입력을 크기가 64인 벡터의 시퀀스로 임베딩
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)

## LSTM을 사용하여 이 벡터들을 하나의 벡터로 인코딩
encoded_text = layers.LSTM(32)(embedded_text)

## 질문도 동일한 과정을 거침
question_input = Input(shape = (None,), dtype = 'int32', name = 'question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

## 인코딩된 질문과 텍스트를 연결
concatenated = layers.concatenate([encoded_text, encoded_question], axis = -1)

## 소프트맥스 분류기를 추가
answer = layers.Dense(answer_vocabulary_size, activation = "softmax")(concatenated)

## 모델 객체를 만들고 2개의 입력과 출력을 주입
model = Model([text_input, question_input], answer)

model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

##### 다중 입력 모델에 데이터 주입하기

In [5]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size = (num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size = (num_samples, max_length))
answers = np.random.randint(0, answer_vocabulary_size, size = num_samples)

answers = to_categorical(answers)

## 리스트 입력을 사용하여 학습합니다
# model.fit([text, question], answers, epochs = 10, batch_size = 128)

## 딕셔너리 입력을 사용하여 학습합니다. 입력 이름을 지정했을때만 사용할 수 있습니다
model.fit({"text": text, "question": question}, answers, epochs = 10, batch_size = 128)

Epoch 1/10
8/8 [==============================] - 2s 10ms/step - loss: 6.2144 - acc: 0.0011
Epoch 2/10
8/8 [==============================] - 0s 9ms/step - loss: 6.1973 - acc: 0.0188
Epoch 3/10
8/8 [==============================] - 0s 9ms/step - loss: 6.1361 - acc: 0.0031
Epoch 4/10
8/8 [==============================] - 0s 10ms/step - loss: 6.0562 - acc: 0.0108
Epoch 5/10
8/8 [==============================] - 0s 9ms/step - loss: 5.9950 - acc: 0.0084
Epoch 6/10
8/8 [==============================] - 0s 9ms/step - loss: 5.9290 - acc: 0.0059
Epoch 7/10
8/8 [==============================] - 0s 9ms/step - loss: 5.8692 - acc: 0.0152
Epoch 8/10
8/8 [==============================] - 0s 11ms/step - loss: 5.7818 - acc: 0.0268
Epoch 9/10
8/8 [==============================] - 0s 9ms/step - loss: 5.6862 - acc: 0.0298
Epoch 10/10
8/8 [==============================] - 0s 9ms/step - loss: 5.6179 - acc: 0.0407


### 다중 출력 모델

같은 방식으로 함수형 API을 사용하여 다중 출력 모델을 만들 수 있습니다.
* 예를 들어 소셜 미디어에서 익명 사용자의 포스트를 입력받아 그 사람의 나이, 성별, 소득 수준을 예측합니다

##### 3개의 출력을 가진 함수형 API 구하기

In [6]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape = (None,), dtype = "int32", name = "posts")
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)

x = layers.Conv1D(128, 5, activation = "relu")(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation = "relu")(x)
x = layers.Conv1D(256, 5, activation = "relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation = "relu")(x)
x = layers.Conv1D(256, 5, activation = "relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation = "relu")(x)

age_prediction = layers.Dense(1, name = "age")(x)

income_prediction = layers.Dense(num_income_groups,
                                 activation = "softmax",
                                 name = "income")(x)

gender_prediction = layers.Dense(1, activation = "sigmoid", name = "gender")(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

<br></br>
![](../images/7-1-3outputs.png)
<br></br>

이런 모델을 훈련하려면 *네트워크 출력*마다 **다른 손실 함수**를 정의해야합니다.
* 나이 예측은 스칼라 회귀 문제이지만 성별 예측은 이진 클래스 문제라 훈련 방식이 다릅니다
* 경사 하강법은 하나의 스칼라 값을 최소화하기 때문에 모델을 훈련하려면 이 손실들을 하나의 값으로 합쳐야 합니다.
* 손실 값을 합치는 가장 간단한 방법은 모두 더하는 것입니다

##### 다중 출력 모델의 컴파일 옵션

In [7]:
model.compile(optimizer = 'rmsprop',
              loss = ['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights = [0.25, 1., 10.])

# model.compile(optimizer = 'rmsprop',
#               loss = {"age": 'mse',
#                       "income": 'categorical_crossentropy',
#                       "gender": 'binary_crossentropy'},
#               loss_weights = {"age": 0.25,
#                               "income": 1.,
#                               "gender": 10.})

##### 다중 출력 모델에 데이터 주입하기

```python
## age_targets, income_targets, gender_targets 가 넘파이 배열이라고 가정
model.fit(posts, [age_targets, income_targets, gender_targets], epochs = 10, batch_size = 64)

## 
model.fit(posts, {'age': age_targets,
                  'income', income_targets,
                  'gender': gender_targets},
          epochs = 10, batch_size = 64)
```

### 층으로 구성된 비순환 유향 그래프

함수형 API를 이용하면 내부 토폴로지가 복잡한 네트워크도 만들 수 있습니다.
* 비순환 유향 그래프 **(directed acyclic graph)**도 만들수 있습니다.
* 다시 말해 이 그래프는 원형을 띌 수 없습니다.
* 텐서 x가 자기 자신을 출력하는 층의 입력이 될 수 없습니다.
* 만들 수 있는 루프는 순환 층의 내부에 있는 것뿐입니다.

#### 입셉션 모듈

**인셉션**은 합성공 신경망에서 인기있는 네트워크 구조입니다.
* 나란히 분리된 가지를 따라 모듈을 쌓아 독립된 작은 네트워크를 구성
* 가장 기본적인 인셉션 모듈 형태는 3~4개의 가지를 가짐
* 1x1 합성곱으로 시작해 3x3 합성곱이 뒤따르고 마지막에 전체 출력 특성이 합쳐짐

이런 구성은 네트워크가 따로따로 공간 특성과 채널 방향의 특성을 학습하도록 돕습니다. 한꺼번에 학습하는 것보다 효과가 더 높습니다.

<br></br>
![](../images/7-1-inception.png)
<br></br>

함수형 API를 이용하여 위의 모듈을 구현해보겠습니다. 입력 x는 4D 텐서라고 가정합니다.

```python
from keras import layers

## 모든 가지는 동일한 스트라이드 (2)를 사용합니다. 출력 크기를 동일하게 만들어 하나로 합치기 위해서입니다
branch_a = layers.Conv2D(128, 1, activation = "relu", strides = 2)(x)

## 이 가지에서는 두번째 합성곱 층에 스트라이드를 적용합니다
branch_b = layers.Conv2D(128, 1, activation = "relu")(x)
branch_b = layers.Conv2D(128, 3, activation = "relu", strides = 2)(branch_b)

## 이 가지에서는 평균 풀링층에 스트라이드를 적용합니다
branch_c = layers.AveragePooling2D(3, strides = 2)(x)
branch_c = layers.Conv2D(128, 3, activation = "relu", strides = 2)(branch_c)

branch_d = layers.Conv2D(128, 1, activation = "relu")(x)
branch_d = layers.Conv2D(128, 3, activation = "relu")(branch_d)
branch_d = layers.Conv2D(128, 3, activation = "relu", strides = 2)(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis = -1)
```

이 인셉션V3 전체 구조는 케라스의 `keras.applications.inception_v3.InceptionV3`에 준비되어있습니다.

#### 잔차 연결

**잔차 연결 (residual connection)**은 많은 네트워크 구조에 있는 *그래프 형태의 네트워크 컴포넌트*입니다.
* 대규모 딥러닝 모델에서 흔히 나타나는 두가지 문제인 *그래디언트 손실*과 *병목*을 해결했습니다.
* 일반적으로 10개 층 이상을 가진 모델에 잔차 연결을 추가하면 도움이 됩니다.
* 잔차 연결은 하위 층의 출력을 상위 층의 입력으로 사용합니다. 
* 순서대로 놓인 네트워크를 질러가는 연결이 만들어 집니다. 
* 하위 층의 출력이 상위 층의 활설화 출력에 연결되는 것이 아니가 더해집니다. 
* 따라서 두 출력의 크기가 동일해야합니다

다음은 특성 맵의 크기가 같을때 원본을 그대로 사용하는 잔차 연결을 구현한 예입니다.

```python
y = layers.Conv2D(128, 3, activation = "relu", padding = "same")(x)
y = layers.Conv2D(128, 3, activation = "relu", padding = "same")(y)
y = layers.Conv2D(128, 3, activation = "relu", padding = "same")(y)

## 원본 x를 출력 특성에 더합니다
y = layers.add([y, x])
```

<br></br>
다음은 특성 맵의 크기가 다를때 선형변환을 사용하여 잔차 연결을 구현한 예입니다.

```python
y = layers.Conv2D(128, 3, activation = "relu", padding = "same")(x)
y = layers.Conv2D(128, 3, activation = "relu", padding = "same")(y)
y = layers.MaxPooling2D(2, strides = 2)(y)

## y와 크기를 맞추기 위해 1x1 합성곱을 사용하여 원본 텐서 x를 다운샘플링합니다
residual = layers.Conv2D(128, 1, strides = 2, padding = "same")(x)

y = layers.add([y, residual])
```

### 층 가중치 공유

함수형 API의 중요한 기능중 하나는 **층 객체를 여러번 재사용**할 수 있다는 것입니다. 층 객체를 두번 호출하면 새로운 층 객체를 만들지 않고 각 호출에 **동일한 가중치를 재사용**합니다. 이런 기능때문에 공유 가지를 가진 모델을 만들 수 있습니다. 이런 가지는 같은 가중치를 공유하고 같은 연산을 수행합니다.

예를 들어 두 문장 사이의 의미가 비슷한지 측정하는 모델을 가정해봅니다.
* 이 모델은 2개의 입력 (비교할 2개의 문장)을 받고 0과 1사이의 점수를 출력합니다.
* 0은 관련 없는 문장이고, 1은 두 문장이 동일하거나 재구성됬다는 것을 의미합니다
* 이런 모델은 Dialog System에서 자연어 질의에 대한 중복 제거같은 예시에 사용됩니다
* 이런 문제에서는 두 입력 시퀀스가 바뀔 수 있습니다
* 의미가 비슷하다는 것은 *대칭적인 관계*이기 때문입니다
* 이런 이유때문에 각 입력 문장을 처리하는 2개의 독립된 모델을 학습하는 것은 이치에 맞지 않습니다.
* **그 대신 하나의 LSTM 층으로 양쪽을 모두 처리하는 것이 좋습니다**
* 이 LSTM 층의 표현은 두 입력에 대해 함께 학습됩니다.
* 이를 **샴 LSTM (Siamese LSTM)** 모델 또는 **공유 LSTM**이라고 부릅니다

```python
from keras import layers
from keras import Input
from keras.models import Model

## LSTM 층 객체 하나를 만듭니다
lstm = layers.LSTM(32)

## 모델의 왼쪽 가지를 구성합니다. 입력은 크기가 128인 벡터의 가변 길이 시퀀스입니다
left_input = Input(shape = (None, 128))
left_output = lstm(left_input)

## 모델의 오른쪽 가지를 구성합니다. 기존 층 객체를 호출하면 가중치가 재사용됩니다.
right_input = Input(shape = (None, 128))
right_output = lstm(right_output)

merged = layers.concatenate([left_output, right_output], axis = -1)
predictions = layers.Dense(1, activation = "sigmoid")(merged)

## 모델 객체를 만들고 훈련합니다. 이런 모델을 훈련하면 LSTM 층의 가중치는 양쪽 입력을 바탕으로 업데이트 합니다.
model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)
```

### 층과 모델

함수형 API에서는 모델을 층처럼 사용할 수 있습니다. 
* 모델을 "커다란 층"으로 생각해도 됩니다. 
* 입력 텐서로 모델을 호출해서 출력 텐서를 얻을 수 있다는 뜻입니다.

`y = model(x)`

모델에서 입력 텐서와 출력 텐서가 여러개이면 텐서의 리스트로 호출합니다

`y1, y2 = model([x1, x2])`

모델 객체를 호출할때 모델의 가중치가 재사용됩니다. 층 객체를 호출할때와 정확히 같습니다
* 객체를 호출하는 것은 항상 그 객체가 가진 학습된 표현을 재사용합니다

간단한 실전 예는 듀얼 카메라에서 입력을 받는 비전 모델입니다.
* 두 카메라가 1인치를 간격을 두고 나란이 있습니다
* 왼쪽과 오른쪽 카메라에서 시각적 특징을 추출하여 합치기 위해 2개의 독립된 모델을 사용할 필요가 없습니다
* 두 입력에 저수준 처리 과정이 공유될 수 있습니다
* 다시 말해 가중치가 같고 동일한 표현을 공유하는 층을 사용합니다

```python
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights = None, include_top = False)

left_input = Input(shape = (250, 250, 3))
right_input = Input(shape = (250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis = -1)
```